# Objectives
YWBAT
- perform a gridsearch in Keras

In [25]:
import warnings
import pandas as pd
import numpy as np
import string
import nltk

from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


import keras
from keras import optimizers
from keras.layers import Dense

from gensim.models import Word2Vec


import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

In [26]:
df_train = pd.read_csv("data/nlp_data_train.csv")
df_test = pd.read_csv("data/nlp_data_test.csv")
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [27]:
df_train.head()

,article,label,processed_article,cleaned_article_string
0,Note: These trial updates are summarized from ...,4,"['note', 'trial', 'updates', 'summarized', 're...",note trial updates summarized reports idaho st...
2,I'm trying to create a button that has both a ...,1,"['trying', 'create', 'button', 'label', 'bitma...",trying create button label bitmap using menubu...
3,"Hi. I've just finished reading S414, and have ...",4,"['hi', ""'ve"", 'finished', 'reading', 's414', '...",hi 've finished reading s414 several questions...
4,\nUnfortunately your phrasing is ambiguous. R...,0,"['unfortunately', 'phrasing', 'ambiguous', 're...",unfortunately phrasing ambiguous re-writing ca...
5,\n\n\tSeAL Team six should have handled it? D...,4,"['seal', 'team', 'six', 'handled', 'delta', 'f...",seal team six handled delta force batf enough ...


In [28]:
# Let's build a tfidf model

In [29]:
tfidf = TfidfVectorizer(max_features=10000)
tfidf_train = tfidf.fit_transform(df_train["cleaned_article_string"], y=df_train['label'])
tfidf_test = tfidf.transform(df_test['cleaned_article_string'])

In [30]:
tfidf_train.shape

In [33]:
model = keras.Sequential()

In [34]:
# first hidden layer
model.add(Dense(input_dim=10000, units=15000, activation='relu'))
# output layter
model.add(Dense(units=5, activation='softmax'))

W0409 13:22:09.250127 140735865734016 deprecation_wrapper.py:119] From /Users/rafael/anaconda3/envs/flatiron-env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0409 13:22:09.275854 140735865734016 deprecation_wrapper.py:119] From /Users/rafael/anaconda3/envs/flatiron-env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [36]:
# encode our y variable
encoder = OneHotEncoder()
y_train_enc = encoder.fit_transform(df_train['label'].values.reshape(-1, 1))
y_test_enc = encoder.transform(df_test['label'].values.reshape(-1, 1))

In [39]:
# build our optimizer
opt = optimizers.Adam()

In [41]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])

In [43]:
history = model.fit(tfidf_train, y_train_enc, batch_size=200, epochs=5, verbose=1)

Epoch 1/5
2756/2756 [==============================] - 40s 14ms/step - loss: 1.0309 - acc: 0.7242
Epoch 2/5
2756/2756 [==============================] - 38s 14ms/step - loss: 0.1155 - acc: 0.9826
Epoch 3/5
2756/2756 [==============================] - 34s 12ms/step - loss: 0.0133 - acc: 0.9985
Epoch 4/5
2756/2756 [==============================] - 33s 12ms/step - loss: 0.0031 - acc: 1.0000
Epoch 5/5
2756/2756 [==============================] - 33s 12ms/step - loss: 0.0017 - acc: 1.0000


In [44]:
y_test_preds = model.predict(tfidf_test)

In [45]:
y_test_preds[:10] # 4, 1, 4

array([[4.18066055e-01, 4.47583152e-05, 8.82277032e-04, 1.94801316e-02,
        5.61526835e-01],
       [3.29944551e-01, 5.19602597e-01, 2.31284034e-02, 1.04378395e-01,
        2.29460038e-02],
       [7.96042877e-06, 3.03912095e-08, 7.22929315e-07, 2.55263858e-05,
        9.99965787e-01],
       [9.99976397e-01, 5.08007304e-07, 7.52068217e-07, 9.92274272e-06,
        1.25066726e-05],
       [2.12732889e-08, 7.86884797e-08, 7.93065080e-09, 9.99999881e-01,
        1.42077381e-08],
       [2.25711480e-01, 1.20060995e-01, 5.23922443e-01, 3.61656025e-02,
        9.41395313e-02],
       [2.44480202e-06, 4.31073931e-06, 9.96799645e-06, 9.99975681e-01,
        7.65553068e-06],
       [1.42257998e-03, 9.84607637e-01, 4.29519685e-03, 3.07999901e-03,
        6.59453822e-03],
       [1.98990833e-02, 1.05127571e-02, 8.11024964e-01, 1.52396858e-01,
        6.16634311e-03],
       [8.34716439e-01, 8.30210629e-04, 2.82891281e-03, 3.06331571e-02,
        1.30991235e-01]], dtype=float32)

In [53]:
# Goal: Find the index with the maximum value -> use argmax

y_test_labels_pred = np.argmax(y_test_preds, axis=1)

In [55]:
report = classification_report(df_test['label'], y_test_labels_pred)
print(report)

              precision    recall  f1-score   support

           0       0.82      0.84      0.83       311
           1       0.95      0.91      0.93       391
           2       0.90      0.95      0.92       389
           3       0.85      0.84      0.85       380
           4       0.82      0.80      0.81       353

    accuracy                           0.87      1824
   macro avg       0.87      0.87      0.87      1824
weighted avg       0.87      0.87      0.87      1824



In [56]:
accuracy_score(df_test['label'], y_test_labels_pred)

0.8706140350877193

# We overfit...whatever...move on...let's tune hyperparameters

In [57]:
from keras.wrappers.scikit_learn import KerasClassifier

In [65]:
def build_model(units1=15000, lr=0.001, layers={}):
    model = keras.Sequential()
    # first hidden layer
    model.add(Dense(input_dim=10000, units=units1, activation='relu'))
    # output layter
    model.add(Dense(units=5, activation='softmax'))
    opt = optimizers.Adam(lr=lr)
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt)
    return model

In [72]:
keras_clf_sklearn = KerasClassifier(build_fn=build_model, verbose=1, batch_size=500)

In [73]:
params = dict(units1 = [15000, 10000], 
              epochs=[3, 5],
              lr=[0.01, 0.001, 1.0])

In [74]:
from sklearn.model_selection import GridSearchCV

In [75]:
gridsearch_cv = GridSearchCV(estimator=keras_clf_sklearn, param_grid=params)

In [ ]:
gridsearch_result = gridsearch_cv.fit(tfidf_train, df_train['label'])

Epoch 1/3
 500/2204 [=====>........................] - ETA: 43s - loss: 1.6099